In [ ]:
import pandas as pd

#表示設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#データの読み込み
df = pd.read_excel('sample_data.xlsx')
df_change = df[['馬連組番','馬連人気順','馬連オッズ','馬単人気順','馬単組番','馬単オッズ']]

#関数の定義
def normalize_pair_key(pair_str):
    """'15-17'のような文字列を受け取り、昇順に並べ替える。文字以外は無視する。"""
    #追加：もしデータが文字列(str)じゃなければ、処理せずにNoneを返す
    if not isinstance(pair_str, str):
        return None
        
    try:
        #文字列を分割し、整数に変換
        h1, h2 = map(int, pair_str.split('-'))
        #昇順のキーを作成
        return f"{min(h1, h2)}-{max(h1, h2)}"
    except (ValueError, AttributeError): # エラーの種類を少し増やしてガードを固めます
        return None
#4.データの分離とキーの作成（変数名を df_change に統一）
df_umaren_vlookup = df_change[['馬連組番', '馬連人気順']].copy()
df_umatan_base = df_change[['馬単組番', '馬単人気順', '馬単オッズ']].copy()

#キーを適用（名前を NormalizedKey に統一）
df_umaren_vlookup.loc[:, 'NormalizedKey'] = df_umaren_vlookup['馬連組番'].apply(normalize_pair_key)
df_umatan_base.loc[:, 'NormalizedKey'] = df_umatan_base['馬単組番'].apply(normalize_pair_key)

#5.馬連テーブルの整理（重複削除）
df_uma_ren_clean = df_umaren_vlookup[['NormalizedKey', '馬連人気順']].dropna().drop_duplicates(subset=['NormalizedKey'])

#6.データの結合（VLOOKUPの代わり）
df_merged = pd.merge(
    df_umatan_base,
    df_uma_ren_clean,
    on='NormalizedKey',
    how='left'
)

print("--- 手順4 完了：馬単組番の横に馬連人気順が反映されました ---")
#結果を表示
print(df_merged[['馬単組番', '馬単人気順', '馬連人気順', '馬単オッズ']].head(30))

#1.人気順位の差 (ΔP) を計算
# 馬連人気順の方が数字が小さい（＝人気がある）のに、馬単では不人気なペアを探す
df_merged['人気順位の差'] = df_merged['馬連人気順'] - df_merged['馬単人気順']

#2.歪みが大きい順（馬連で人気だが馬単で売れていない順）に並べ替え
#期待値が高い候補を上位に表示
df_analysis = df_merged.sort_values(by='人気順位の差', ascending=False)

print("--- 手順5 完了：オッズの歪み分析結果 ---")
#「馬単オッズの隣に歪み(人気順位の差)」を表示する配置
print(df_analysis[['馬単組番', '馬連人気順', '馬単人気順', '馬単オッズ', '人気順位の差']].head(20))

#1.表示する項目を整理し、新しい順位をつける
#「人気順位の差」が大きい順に並べ替え、インデックス（左の番号）を振り直す
df_final = df_analysis[['馬単組番', '馬連人気順', '馬単人気順', '馬単オッズ', '人気順位の差']].reset_index(drop=True)

#2.列名をより分かりやすくリネーム
df_final.columns = ['馬単ペア', '馬連人気', '馬単人気', 'オッズ', '歪み(ΔP)']

#3.綺麗な表形式で表示（Jupyterの機能を使用）
#head(20)で上位20件を表示
print("【分析結果：オッズの歪み上位】")
display(df_final.head(30))

